<a href="https://colab.research.google.com/github/ishitaa27/BooleanRetrievalSystem/blob/main/IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

import os,glob
import string
import time

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#removing stop words

stop_words=nltk.corpus.stopwords.words('english')


def preprocess(file1):
  line = file1.read()
  words_r = line.split()
  
  # remove punctuation from each word
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in words_r]

  words = [word.lower() for word in stripped]
  #words=nltk.word_tokenize(stri)

  #print(len(words))
  #print(words)
  final_words=[]
  for r in words:
      if not r in stop_words:
        final_words.append(r)

  #print(len(final_words))
  #print(final_words)
  return final_words


In [ ]:
#Lemmatization
def lemmatization(final_words):
  
  #an instance of Word Net Lemmatizer
  lemmatizer = WordNetLemmatizer()   

  lemmatized_words = [lemmatizer.lemmatize(word) for word in final_words]  
  unique_words=set(lemmatized_words)
  return unique_words

In [ ]:
#generating an inverted index
inverted_index={}
#set of all documents, required for processing boolean queries
universal = []

#words is the list of words in the text file after preprocesing
#doc is the name of the text file
def generate_inverted_index(words,doc):
  if doc not in universal:
    universal.append(doc)
  for word in words:
    if word not in inverted_index.keys():
      inverted_index[word]=[doc]
    elif doc not in inverted_index[word]:
      inverted_index[word].append(doc)



In [ ]:
#function that removes the IO wrapper and returns just the path of the file along with the name
def extractPath(winnie):
    piggy = str(winnie)

    foff1 ="<_io.TextIOWrapper name='"
    foff2 ="' mode='r' encoding='UTF-8'>"

    piggy = piggy.replace(foff1, "")
    piggy = piggy.replace(foff2, "")

    return piggy

In [ ]:

#Folder path is the root directory which contains the documents needed for processing
folder_path='/content/drive/MyDrive/Assignment'
func_start_time = time.time()
preprocess_time = 0
for filename in glob.glob(os.path.join(folder_path,'*.txt')):
  with open(filename,'r') as f:
    start_time = time.time()
    #sending the files for preprocessing
    tokens=preprocess(f)
    #cleaned tokens are sent for lemmatization
    final_tokens=lemmatization(tokens)
    preprocess_time += time.time() - start_time
    #to extract the path of the file
    file_n=extractPath(f)
    #generates inverted index taking name of the document as a parameter and tokens as another
    generate_inverted_index(final_tokens,file_n.split('/')[-1])

print("time for preprocessing = %.2f seconds" % (preprocess_time))
print("time for indexing = %.2f seconds" % (time.time() - func_start_time))
complete_words=list(inverted_index.keys())

  

time for preprocessing = 5.46 seconds
time for indexing = 5.65 seconds


In [ ]:
#del inverted_index['']

print(len(complete_words))
print(len(inverted_index))
print(complete_words)
print(inverted_index[''])
print(len(inverted_index['']))
print(complete_words[1])
print(complete_words[2])
print(nltk.edit_distance(complete_words[1],complete_words[2]))


26160
26160
['', 'plucked', 'daunted', 'rampant', 'grieve', 'islander', 'soar', 'lingring', 'asketh', 'anything', 'neer', 'tour', 'fifteen', 'maintain', 'chance', 'mighty', 'westminster', 'besom', 'thank', 'grovel', 'erect', 'report', 'removed', 'droop', 'annoy', 'hoise', 'crept', 'agent', 'warm', 'bloodconsuming', 'baby', 'culpable', 'heartoffending', 'loved', 'tomb', 'built', 'conquered', 'played', 'case', 'evil', 'repair', 'council', 'joy', 'bud', 'act', 'premised', 'oughtst', 'dew', 'soil', 'anjou', 'sharpquilled', 'confederate', 'alarum', 'journey', 'study', 'passenger', 'name', 'hamper', 'flesh', 'throatsfor', 'dry', 'stood', 'manacle', 'firmly', 'dick', 'fullness', 'studied', 'knave', 'forlorn', 'confirms', 'retreat', 'thrive', 'eat', 'parching', 'raise', 'rome', 'load', 'cumberland', 'loudhowling', 'misty', 'cardinal', 'shakespeare', 'presence', 'exhort', 'thereof', 'thorn', 'bearherd', 'hautboy', 'prophet', 'bandogs', 'cause', 'clarence', 'consent', 'rout', 'heave', 'accuse', 

In [ ]:
#returns the word that has least edit distance with given word
#if multiple words have same least edit distance, the word occurring in more number of documents is returned
def spell_correction(tokens,word):
  edit_dist=10000000
  for token in tokens:
    dist=nltk.edit_distance(token,word)
    if dist<edit_dist:
      ans=token
      edit_dist=dist
    elif dist==edit_dist:
      if len(inverted_index[token])>len(inverted_index[ans]):
        ans=token
  return ans

In [ ]:
#Function to find set of documents present in both list1 and list2
def find_and(list1,list2):
  result=list(set(list1) & set(list2))
  return result

#function to find list of documents not present in list1
def find_not(list1):
  result=list(set(universal) - set(list1))
  return result

#function to find list of documents present in either list1 or list2
def find_or(list1,list2):
  list1.extend(list2)
  result=list(set(list1))
  return result


In [ ]:
#function that returns all the cyclic permutations of given string
def permute(input):
    tmp=input
    s=len(input)+1
    input+='$'+input
    res=[]
    for x in range(s):
        #print(input[x:x+s])
        res.append(input[x:x+s])
     
    return res

In [ ]:
#contains all the cyclic permutations of all the words mapped to the unrotated word
permuterm_index={}

#creates permuterm index for all the words in the list represented by tokens
def permuterm(tokens):
  for token in tokens:
    res=permute(token)
    for item in res:
      permuterm_index[item]=token

permuterm(complete_words)

print(len(permuterm_index))

228654


In [ ]:
#Function that rotates the query for smooth searching
#Appending the '$' sign at the end and rotating it so the '*' is at the end for searching
def rotate_query(query):
  query+='$'
  length=len(query)
  for d in range(length):
    Rfirst = query[0 : len(query)-d]
    Rsecond = query[len(query)-d : ]
    #creating a permutation by rotating right each time
    rotated_query=Rsecond+Rfirst
    #returning only the rotation that has '*' as last character
    if rotated_query[-1]=='*':
      res=rotated_query
      break
  #return rotated query after stripping the '*'
  return res[:-1]

In [ ]:
#Does prefix match for given query against all the words in permuterm_index
#returns those words whose prefix matches with the query
def prefix_match(query):
  res=[]
  for token in permuterm_index:
    #checks whether the token starts with the query
    if token.startswith(query):
      #print(token)
      res.append(permuterm_index[token])
  #set contains unique words which satisfy the wildcard 
  final_res=set(res)
  #print(res)
  return final_res


In [ ]:
#Handles the wildcard queries
def wildcard_query(query):
  #to rotate the query to suit matching
  query=rotate_query(query)
  #list containing words satisfying wildcard
  matched_words=prefix_match(query)
  #docs contains list of documents containing these words
  docs=[]
  for token in matched_words: 
    for doc in inverted_index[token]:
      docs.append(doc)
  
  #set containing the unique documents that contain the words satisfying wildcard queries
  final_docs=set(docs)
  return final_docs


In [ ]:
#Function to compute boolean queries
def boolean_query(query):
  #query=input("Enter your query:")
  #converts query to lowercase
  query = query.lower()
  #stores each argument of query in a list
  args = list(query.split(' '))

  bool_q = ['and', 'or', 'not']
  q_size = len(args)
  wildcard=[]
  
  #spell correction for wrongly spelt words
  for i in range(len(args)):
    if (args[i] not in bool_q) and (args[i] not in complete_words):
      args[i]=spell_correction(complete_words,args[i])

  #Initializing ans and checking if query is valid
  if args[0]=='not' and q_size!=1 and args[1] not in bool_q:
    ans = find_not(inverted_index[args[1]])
    i=2
  elif args[0] not in bool_q:
    ans = inverted_index[args[0]]
    i=1
  else:
    print("Invalid Query")
    return []
  
  #Loop to find solutions to all queries
  while i<q_size-1:
    #Query of form NOT A
    if args[i]=='not' and args[i+1] not in bool_q:
      ans = find_not(inverted_index[args[i+1]])
      i+=2
    #Query of form A AND B
    elif args[i]=='and' and args[i+1]!='not':
      ans = find_and(ans,inverted_index[args[i+1]])
      i+=2
    #Query of form A AND NOT B
    elif args[i]=='and' and i<q_size-2:
      list1 = find_not(inverted_index[args[i+2]])
      ans = find_and(ans,list1)
      i+=3
    #Query of form A OR B
    elif args[i]=='or' and args[i+1]!='not':
      ans = find_or(ans, inverted_index[args[i+1]])
      i+=2
    #Query of form A OR NOT B
    elif args[i]=='or' and i<q_size-2:
      list1 = find_not(inverted_index[args[i+2]])
      ans = find_or(ans,list1)
      i+=3
    #Query is in invalid format
    else:
      print("Invalid Query")
      ans = []
      break
    #print(i)
  return ans



In [ ]:
print("You have 3 choices.")
print("1.Wildcard query that returns the list of documents the words satisfying the query is present in.")
print("2.Boolean Queries that prints the documents satisfying the query. It also corrects the spelling if a word is spelt wrong.")
print("3.finding the word that is closest to the list of words present in the documents")
num=input("Enter your choice")

if num=="1":
  user_query=input("Enter your query:")
  result=wildcard_query(user_query)
  print(result)
elif num=="2":
  user_query=input("Enter your query:")
  results=boolean_query(user_query)
  if len(results)>0:
    print(results)
    print(len(results))
  else:
    print("No document satisfies given query")
elif num=="3":
  user_query=input("Enter the word")
  if user_query not in complete_words:
    result=spell_correction(complete_words,user_query)
    print(result)
  else:
    print(user_query)


You have 3 choices.
1.Wildcard query that returns the list of words and the documents the word is present in.
2.Boolean Queries that prints the documents satisfying the query. It also corrects the spelling if a word is spelt wrong.
3.finding the word that is closest to the list of words present in the documents
Enter your choice1
Enter your query:m*n
{'julius-caesar_TXT_FolgerShakespeare.txt', 'othello_TXT_FolgerShakespeare.txt', 'king-john_TXT_FolgerShakespeare.txt', 'alls-well-that-ends-well_TXT_FolgerShakespeare.txt', 'the-merry-wives-of-windsor_TXT_FolgerShakespeare.txt', 'the-merchant-of-venice_TXT_FolgerShakespeare.txt', 'henry-v_TXT_FolgerShakespeare.txt', 'timon-of-athens_TXT_FolgerShakespeare.txt', 'hamlet_TXT_FolgerShakespeare.txt', 'venus-and-adonis_TXT_FolgerShakespeare.txt', 'twelfth-night_TXT_FolgerShakespeare.txt', 'a-midsummer-nights-dream_TXT_FolgerShakespeare.txt', 'richard-ii_TXT_FolgerShakespeare.txt', 'much-ado-about-nothing_TXT_FolgerShakespeare.txt', 'the-taming-